In [10]:
import numpy as np
import pandas as pd

In [11]:
class QueueMM2():
    def __init__(self, lbd, mu): 
        
        self.lbd = lbd                        #каждые lbd минут поступает новое требование
        self.mu = mu                          #сервисное время 1/mu
        
        self.clock = 0.0                      #simulation clock
        self.num_arrivals = 0                 #total number of arrivals
        self.next_arriv_time = self.gen_arrival_time()   #time of next arrival
        
        self.leaving_time1 = float('inf')     #departure time from server 1
        self.leaving_time2 = float('inf')     #departure time from server 2
        
        self.S1_state = 0                     #current state of server1 (0-free, 1-busy)
        self.S2_state = 0                     #current state of server2 (0-free, 1-busy)
        
        self.num_of_departures1=0             #number of customers served by server 1  
        self.num_of_departures2=0             #number of customers served by server 2 
        
        self.sum_serv_times1=0                #sum of service times by server 1
        self.sum_serv_times2=0                #sum of service times by server 2
        
        self.queue_length = 0                 #current number in queue
        self.lost_customers = 0               #customers who left without service
        
    def time_adv(self):                                                       
        t_next_event = min(self.next_arriv_time, self.leaving_time1, self.leaving_time2)  
        self.clock = t_next_event      
        if (self.next_arriv_time < self.leaving_time1) and (self.next_arriv_time < self.leaving_time2):
            self.Arrival()
        elif (self.next_arriv_time > self.leaving_time1) and (self.leaving_time1 < self.leaving_time2):
            self.Server1()
        else:
            self.Server2()
            
    def Arrival(self):              
        self.num_arrivals += 1

        if (self.queue_length == 0):                      #schedule next departure or arrival depending on state of servers
            if (self.S1_state==1 and self.S2_state==1):        #both servers are busy
                self.queue_length += 1
                self.next_arriv_time = self.clock + self.gen_arrival_time()
            elif (self.S1_state==0 and self.S2_state==0):      #both servers are free
                if (np.random.choice([0,1])==1):
                    self.S1_state = 1
                    self.service_time1 = self.gen_service_time()
                    self.sum_serv_times1 += self.service_time1
                    self.leaving_time1 = self.clock + self.service_time1
                    self.next_arriv_time = self.clock + self.gen_arrival_time()
                else:
                    self.S2_state = 1
                    self.service_time2 = self.gen_service_time()
                    self.sum_serv_times2 += self.service_time2
                    self.leaving_time2 = self.clock + self.service_time2
                    self.next_arriv_time = self.clock + self.gen_arrival_time()
            elif (self.S1_state==0 and self.S2_state==1):       #if server 2 is busy customer goes to server 1
                self.S1_state = 1
                self.service_time1 = self.gen_service_time()
                self.sum_serv_times1 += self.service_time1
                self.leaving_time1 = self.clock + self.service_time1
                self.next_arriv_time = self.clock + self.gen_arrival_time()
            else:                                               #if server 1 is busy customer goes to server 2
                self.S2_state = 1
                self.service_time2 = self.gen_service_time()
                self.sum_serv_times2 += self.service_time2
                self.leaving_time2 = self.clock + self.service_time2
                self.next_arriv_time = self.clock + self.gen_arrival_time()
        elif (self.queue_length<5 and self.queue_length>=1): #if queue length is less than 5 generate next arrival and make customer join queue
            self.queue_length += 1 
            self.next_arriv_time = self.clock + self.gen_arrival_time()
        elif (self.queue_length>=5):                          #if queue length is >=5 equal prob to leave or stay
            if (np.random.choice([0,1])==1): 
                self.queue_length += 1 
                self.next_arriv_time = self.clock + self.gen_arrival_time()
            else:
                self.lost_customers += 1
                
    def Server1(self):                #departure from server 2
        self.num_of_departures1 += 1
        if (self.queue_length>0):
            self.service_time1 = self.gen_service_time()
            self.sum_serv_times1 += self.service_time1
            self.leaving_time1 = self.clock + self.service_time1
            self.queue_length -= 1
        else:
            self.leaving_time1 = float('inf') 
            self.S1_state = 0                  
    
    def Server2(self):                #departure from server 1
        self.num_of_departures2 += 1
        if (self.queue_length>0):
            self.service_time2 = self.gen_service_time()
            self.sum_serv_times2 += self.service_time2
            self.leaving_time2 = self.clock + self.service_time2
            self.queue_length -= 1
        else:
            self.leaving_time2 = float('inf')
            self.S2_state = 0
            
    def gen_arrival_time(self):            #function to generate arrival times using poisson distribution
        return (np.random.poisson(self.lbd))
    
    def gen_service_time(self):            #function to generate service time for servers using exponential distribution
        return (np.random.exponential(1./self.mu))
    

In [15]:
s=QueueMM2(5,0.1)
df=pd.DataFrame(columns=['Average interarrival time',
                         'Average service time server1', 'Average service time server2',
                         'Lost Customers', 'Queue length'])

for i in range(100):
    np.random.seed(i)
    s.__init__(5,0.1)
    while s.clock <= 300:
        s.time_adv() 
    a = pd.Series([s.clock/s.num_arrivals, 
                   s.sum_serv_times1/s.num_of_departures1, s.sum_serv_times2/s.num_of_departures2,
                   s.lost_customers, s.queue_length], index=df.columns)
    df = df.append(a, ignore_index=True)   
    
df.to_excel('results.xlsx')   

In [16]:
df

,Average interarrival time,Average service time server1,Average service time server2,Lost Customers,Queue length
0,4.046302,17.699580,8.322081,17.0,3.0
1,3.786916,11.087224,9.221781,19.0,2.0
2,4.983607,9.292104,8.887190,0.0,3.0
3,2.980392,10.489611,9.886959,41.0,7.0
4,3.948052,14.050618,8.537161,15.0,6.0
...,...,...,...,...,...
95,3.377778,11.136463,11.035096,31.0,5.0
96,5.101695,8.726856,5.015663,0.0,0.0
97,5.016667,11.770778,9.143073,3.0,2.0
98,3.974359,9.353806,11.000840,16.0,5.0
